# Notebook pour système de recommandation de films

In [1]:
# Import des librairies utiles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.model_selection import train_test_split # Split des jeux de tests
from sklearn.linear_model import LinearRegression # Regression linéaire
from sklearn.linear_model import LogisticRegression # Regression logistique (demerde tes couilles)
from sklearn.neighbors import KNeighborsClassifier # K plus proche copains
from sklearn.neighbors import NearestNeighbors # Voisin proche (?)
from sklearn.tree import DecisionTreeClassifier # arbre de décision classifiant (données non numériques)
from sklearn.tree import DecisionTreeRegressor # arbre de décision regressif (données numériques)
from sklearn.metrics import confusion_matrix # Matrice de confusion
from sklearn.metrics import classification_report # Rapport detaillé des metriques de performance d'un modele après son entrainement et sa prediction
from sklearn.preprocessing import StandardScaler # Standardise les données
from sklearn.model_selection import cross_val_score  # permet de balayer tout le dataset avec des découpages

In [2]:
pd.set_option("display.max_columns", None)

## Step 1 : import des datasets préalablement cleanés utilisées pour les KPIS


In [3]:
tmdbFr = pd.read_csv("/home/skikk/Documents/Formation data/Projet 2/tmdbFRClean.csv")
d_a_tmdbFr = pd.read_csv("/home/skikk/Documents/Formation data/Projet 2/directors_actors_tmdb_merge.csv")
# Pas utile ici : d_a_2000 = pd.read_csv("/home/skikk/Documents/Formation data/Projet 2/directors_and_actors_fr_rating(depuis2000).csv")
movieFrance = pd.read_csv("/home/skikk/Documents/Formation data/Projet 2/movieFranceClean(depuis2000).csv")
# Pas utile ici : indicateurCompagnie = pd.read_csv("/home/skikk/Documents/Formation data/Projet 2/indicateurs_par_compagnie.csv")

In [4]:
# Pas utile ici : display(f"Infos sur la table tmdbFr {tmdbFr.info()}")
display(f"\nInfos sur la table d_a_tmdbFr {d_a_tmdbFr.info()}")
# Pas utile ici : display(f"\nInfos sur la table d_a_2000 {d_a_2000.info()}")
display(f"\nInfos sur la table movieFrance {movieFrance.info()}")
# Pas utile ici : display(f"\nInfos sur la table indicateurCompagnie {indicateurCompagnie.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127406 entries, 0 to 127405
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    127406 non-null  int64  
 1   tconst                        127406 non-null  object 
 2   nconst                        127406 non-null  object 
 3   category                      127406 non-null  object 
 4   characters                    127406 non-null  object 
 5   primaryName                   127406 non-null  object 
 6   birthYear                     127406 non-null  object 
 7   deathYear                     127406 non-null  object 
 8   primaryProfession             126869 non-null  object 
 9   knownForTitles                127406 non-null  object 
 10  averageRating                 127406 non-null  float64
 11  numVotes                      127406 non-null  int64  
 12  backdrop_path                 110657 non-nul

'\nInfos sur la table d_a_tmdbFr None'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29950 entries, 0 to 29949
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       29950 non-null  int64  
 1   tconst           29950 non-null  object 
 2   titleType        29950 non-null  object 
 3   primaryTitle     29950 non-null  object 
 4   originalTitle    29950 non-null  object 
 5   isAdult          29950 non-null  int64  
 6   startYear        29950 non-null  int64  
 7   runtimeMinutes   29950 non-null  int64  
 8   genres           29950 non-null  object 
 9   Genre1           29950 non-null  object 
 10  Genre2           19924 non-null  object 
 11  Genre3           11773 non-null  object 
 12  averageRating    29950 non-null  float64
 13  numVotes         29950 non-null  int64  
 14  ordering         29950 non-null  int64  
 15  title            29950 non-null  object 
 16  region           29950 non-null  object 
 17  language    

'\nInfos sur la table movieFrance None'

In [5]:
# Display dataframes
display(d_a_tmdbFr, movieFrance)

,Unnamed: 0,tconst,nconst,category,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,averageRating,numVotes,backdrop_path,budget,id,imdb_id,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,production_companies_country
0,0,tt0308078,nm0258477,director,\N,Jean Epstein,1897,1953,"director,writer,production_manager","tt0200846,tt0018770,tt0013933,tt0191246",6.1,16,NaN,0,381426,tt0308078,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,['Ciné-Liberté'],['']
1,1,tt0283832,nm0201638,actress,"[""Mamy""]",Danielle Darrieux,1917,2017,"actress,soundtrack,music_department","tt0027951,tt0044314,tt0062873,tt0283832",7.0,33935,/nL9x7wZKMBhCIMxZioKIFXKA9Av.jpg,8000000,1958,tt0283832,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L...","['FR', 'FR', 'FR', 'FR', '', 'FR', 'IT', 'IT',..."
2,2,tt0283832,nm0000366,actress,"[""Gaby""]",Catherine Deneuve,1943,\N,"actress,soundtrack,producer","tt0061395,tt0059646,tt0283832,tt0168629",7.0,33935,/nL9x7wZKMBhCIMxZioKIFXKA9Av.jpg,8000000,1958,tt0283832,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L...","['FR', 'FR', 'FR', 'FR', '', 'FR', 'IT', 'IT',..."
3,3,tt0283832,nm0000272,actress,"[""Pierrette""]",Fanny Ardant,1949,\N,"actress,director,writer","tt0348853,tt0283832,tt0086551,tt0127536",7.0,33935,/nL9x7wZKMBhCIMxZioKIFXKA9Av.jpg,8000000,1958,tt0283832,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L...","['FR', 'FR', 'FR', 'FR', '', 'FR', 'IT', 'IT',..."
4,4,tt0283832,nm0000322,actress,"[""Louise""]",Emmanuelle Béart,1963,\N,"actress,soundtrack","tt0117060,tt0113947,tt0109731,tt0283832",7.0,33935,/nL9x7wZKMBhCIMxZioKIFXKA9Av.jpg,8000000,1958,tt0283832,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L...","['FR', 'FR', 'FR', 'FR', '', 'FR', 'IT', 'IT',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127401,127401,tt9617464,nm10408328,director,\N,Jennifer Trainer,\N,\N,"director,writer,producer",tt9617464,7.5,31,NaN,0,576552,tt9617464,en,0.600,/5L576j6jxvWmdgA94xfuM7gCoGz.jpg,2019-03-10,0,['en'],Released,[],[]
127402,127402,tt9654050,nm10518988,director,\N,Joanna Reposi Garibaldi,\N,\N,"director,writer,producer",tt9654050,6.6,332,/2jauWa6t5TW2uszCyBxIH2Mmv2Y.jpg,0,577080,tt9654050,es,1.398,/dqXbYgaXOtZxIRNE6dS9Nynvw2r.jpg,2019-09-05,0,['es'],Released,['Solita Producciones'],['CL']
127403,127403,tt9658178,nm9466043,director,\N,Suhaib Gasmelbari,1979,\N,"director,writer,cinematographer","tt7710684,tt9658178",7.4,845,/z71TRHVwaL2j7TJVlM2TYswhkmA.jpg,0,577054,tt9658178,en,3.063,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019-12-18,0,"['ar', 'en']",Released,"['Agat Films & Cie / Ex Nihilo', 'Made In Germ...","['FR', 'DE', 'TD', 'QA', 'SD', 'FR']"
127404,127404,tt9691896,nm0663605,director,\N,Denis Parrot,\N,\N,"editorial_department,editor,sound_department","tt0144201,tt6814150,tt0236638,tt9691896",7.2,113,/pnW9lRWSgWNGGnUtUUATkepK54Z.jpg,0,534369,tt9691896,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,"['Dryades Films', 'Upside Films']","['FR', 'FR']"


,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,ordering,title,region,language,isOriginalTitle
0,0,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,2,Histoire de la guerre civile,FR,\N,0
1,1,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,11,Kate et Léopold,FR,\N,0
2,2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama,Drama,NaN,NaN,6.4,183,7,El Tango del Viudo y Su Espejo Deformante,FR,\N,0
3,3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama,Drama,NaN,NaN,6.7,7906,15,De l'autre côté du vent,FR,\N,0
4,4,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,94,Horror,Horror,NaN,NaN,5.9,1761,6,L'incarnation du démon,FR,\N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,29945,tt9911196,movie,The Marriage Escape,De beentjes van Sint-Hildegard,0,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,8,De Beentjes van Sint-Hildegard,FR,\N,0
29946,29946,tt9913084,movie,Diabolik sono io,Diabolik sono io,0,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,2,Diabolik sono io,FR,\N,0
29947,29947,tt9913936,movie,Paradise,Paradise,0,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,6,Le paradis de Diego,FR,\N,0
29948,29948,tt9915790,movie,Bobbyr Bondhura,Bobbyr Bondhura,0,2019,106,Family,Family,NaN,NaN,7.2,40,3,Bobbyr Bondhura,FR,\N,0


## Step 2 : retravailler les tables pour pouvoir les exploiter dans le système

### Drop les colonnes redondantes (s'il y en a)

In [6]:
# Pour movieFrance, il y a des filtres qui ont été effectués dans le notebook d'exploration, donc ces colonnes ne seront pas utiles. 
# Genre 1, 2 et 3 non plus car nous allons préférer faire un explode et un get dummies par la suite
movieFranceV2 = movieFrance.drop(columns=["Unnamed: 0", "titleType", "primaryTitle", "originalTitle", "isAdult", "region", "language", "isOriginalTitle", "Genre1", "Genre2", "Genre3"])

In [7]:
# Afficher la v2 de movieFrance
movieFranceV2

,tconst,startYear,runtimeMinutes,genres,averageRating,numVotes,ordering,title
0,tt0013274,2021,94,Documentary,6.8,62,2,Histoire de la guerre civile
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",6.4,88199,11,Kate et Léopold
2,tt0062336,2020,70,Drama,6.4,183,7,El Tango del Viudo y Su Espejo Deformante
3,tt0069049,2018,122,Drama,6.7,7906,15,De l'autre côté du vent
4,tt0082328,2008,94,Horror,5.9,1761,6,L'incarnation du démon
...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",7.5,3207,8,De Beentjes van Sint-Hildegard
29946,tt9913084,2019,75,Documentary,6.7,51,2,Diabolik sono io
29947,tt9913936,2019,135,"Crime,Drama",7.4,62,6,Le paradis de Diego
29948,tt9915790,2019,106,Family,7.2,40,3,Bobbyr Bondhura


In [8]:
# Pour d_a_tmdbFr
# Afficher le nombre de valeurs manquantes dans la table d_a_tmdbFr
d_a_tmdbFr.isna().sum()

Unnamed: 0                          0
tconst                              0
nconst                              0
category                            0
characters                          0
primaryName                         0
birthYear                           0
deathYear                           0
primaryProfession                 537
knownForTitles                      0
averageRating                       0
numVotes                            0
backdrop_path                   16749
budget                              0
id                                  0
imdb_id                             0
original_language                   0
popularity                          0
poster_path                      2272
release_date                      658
revenue                             0
spoken_languages                    0
status                              0
production_companies_name           0
production_companies_country    49471
dtype: int64

In [9]:
# Nous allons donc supprimer les colonnes production_compagnies_countruy, unnamed, backdrop_path, car ces colonnes présente un nombre important de valeurs manquantes et unnamed est une colonne d'indexation
# D'autres colonnes comme nconst et primary profession ne sont pas utile ici. imdb_id à les mêmes valeurs que tconst, donc à supprimer aussi
d_a_tmdbFr_drop = d_a_tmdbFr.drop(columns=["Unnamed: 0", "nconst", "backdrop_path", "primaryProfession", "imdb_id", "production_companies_country", "characters", "birthYear", "deathYear", "id", "knownForTitles"])

In [10]:
# Afficher d_a_tmdbfr_drop
d_a_tmdbFr_drop

,tconst,category,primaryName,averageRating,numVotes,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name
0,tt0308078,director,Jean Epstein,6.1,16,0,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,['Ciné-Liberté']
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L..."
2,tt0283832,actress,Catherine Deneuve,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L..."
3,tt0283832,actress,Fanny Ardant,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L..."
4,tt0283832,actress,Emmanuelle Béart,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,"['France 2 Cinéma', 'Fidélité Productions', 'L..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127401,tt9617464,director,Jennifer Trainer,7.5,31,0,en,0.600,/5L576j6jxvWmdgA94xfuM7gCoGz.jpg,2019-03-10,0,['en'],Released,[]
127402,tt9654050,director,Joanna Reposi Garibaldi,6.6,332,0,es,1.398,/dqXbYgaXOtZxIRNE6dS9Nynvw2r.jpg,2019-09-05,0,['es'],Released,['Solita Producciones']
127403,tt9658178,director,Suhaib Gasmelbari,7.4,845,0,en,3.063,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019-12-18,0,"['ar', 'en']",Released,"['Agat Films & Cie / Ex Nihilo', 'Made In Germ..."
127404,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,"['Dryades Films', 'Upside Films']"


### Explode certaines colonnes pertinentes

In [11]:
# Explode de la colonne genres de la table movieFranceV2 pour faire un get_dummies ensuite
movieFranceV2["genres"] = movieFranceV2["genres"].str.split(",")

In [12]:
# Explode de la colonne genres
movieFranceV2Explode = movieFranceV2.explode("genres")

In [13]:
# Afficher la nouvelle base
movieFranceV2Explode

,tconst,startYear,runtimeMinutes,genres,averageRating,numVotes,ordering,title
0,tt0013274,2021,94,Documentary,6.8,62,2,Histoire de la guerre civile
1,tt0035423,2001,118,Comedy,6.4,88199,11,Kate et Léopold
1,tt0035423,2001,118,Fantasy,6.4,88199,11,Kate et Léopold
1,tt0035423,2001,118,Romance,6.4,88199,11,Kate et Léopold
2,tt0062336,2020,70,Drama,6.4,183,7,El Tango del Viudo y Su Espejo Deformante
...,...,...,...,...,...,...,...,...
29947,tt9913936,2019,135,Crime,7.4,62,6,Le paradis de Diego
29947,tt9913936,2019,135,Drama,7.4,62,6,Le paradis de Diego
29948,tt9915790,2019,106,Family,7.2,40,3,Bobbyr Bondhura
29949,tt9916362,2020,92,Drama,6.4,5649,10,Les sorcières d'Akelarre


In [14]:
# Explode des colonnes utiles dans d_a_tmdbFr_drop
# Importer literal_eval pour exploser les colonnes production_compagnies_name
from ast import literal_eval

d_a_tmdbFr_drop["production_companies_name"] = d_a_tmdbFr_drop["production_companies_name"].apply(literal_eval)

In [15]:
# Exploser la colonne production_companies_name
d_a_tmdbFr_explode = d_a_tmdbFr_drop.explode("production_companies_name")

In [16]:
# Afficher la table explosée
d_a_tmdbFr_explode

,tconst,category,primaryName,averageRating,numVotes,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name
0,tt0308078,director,Jean Epstein,6.1,16,0,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,Ciné-Liberté
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Fidélité Productions
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Le Studio Canal+
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,CNC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127403,tt9658178,director,Suhaib Gasmelbari,7.4,845,0,en,3.063,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019-12-18,0,"['ar', 'en']",Released,SFG
127403,tt9658178,director,Suhaib Gasmelbari,7.4,845,0,en,3.063,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019-12-18,0,"['ar', 'en']",Released,Vidéo de Poche
127404,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Dryades Films
127404,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Upside Films


### Get dummies

1 er test sur les tables explodées

In [17]:
# Get dummies la colonne genres
# movieFrance_dummy = pd.get_dummies(movieFranceV2Explode, columns=["genres"])

In [18]:
# Enlever le préfixe "genres_" sur le nom de chaque colonne provenant du get_dummies
# movieFrance_dummy.columns = movieFrance_dummy.columns.str.replace("genres_", "")

In [19]:
# Reset de l'index
# movieFrance_dummy = movieFrance_dummy.reset_index(drop=True)

In [20]:
# Afficher la table movieFrance_dummy
# movieFrance_dummy

In [21]:
# Get dummies la colonne production_companies_name, primaryName et category de d_a_tmdbFr_explode
# d_a_tmdbFr_dummy = pd.get_dummies(d_a_tmdbFr_explode, columns=["category"])

In [22]:
# d_a_tmdbFr_dummy = pd.get_dummies(d_a_tmdbFr_dummy, columns=["production_companies_name"])

In [23]:
# d_a_tmdbFr_dummy = pd.get_dummies(d_a_tmdbFr_dummy, columns=["primaryName"], sparse=True)

2 ème test sur les tables non explodées

In [24]:
# Avec une autre méthode, sans utiliser l'explode avant
genres_dummies = movieFrance["genres"].str.get_dummies(sep=",")

In [25]:
# Concatener les genres_dummies avec la table de base
movieFrance_full = pd.concat([movieFrance, genres_dummies], axis=1)

In [26]:
# Drop des colonnes inutiles
movieFrance_full_drop = movieFrance_full.drop(columns=["Unnamed: 0", "titleType", "primaryTitle", "originalTitle", "isAdult", "region", "language", "isOriginalTitle", "ordering"])

In [27]:
movieFrance_full_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29950 entries, 0 to 29949
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          29950 non-null  object 
 1   startYear       29950 non-null  int64  
 2   runtimeMinutes  29950 non-null  int64  
 3   genres          29950 non-null  object 
 4   Genre1          29950 non-null  object 
 5   Genre2          19924 non-null  object 
 6   Genre3          11773 non-null  object 
 7   averageRating   29950 non-null  float64
 8   numVotes        29950 non-null  int64  
 9   title           29950 non-null  object 
 10  Action          29950 non-null  int64  
 11  Adult           29950 non-null  int64  
 12  Adventure       29950 non-null  int64  
 13  Animation       29950 non-null  int64  
 14  Biography       29950 non-null  int64  
 15  Comedy          29950 non-null  int64  
 16  Crime           29950 non-null  int64  
 17  Documentary     29950 non-null 

In [28]:
# tmdbFr["production_companies_name"].str.get_dummies(sep=", ")

In [29]:
# tmdbFr["production_companies_name"].apply(lambda row : ''.join(row))

### Merge

In [30]:
# Merge sans dummies pour tester
full_data = d_a_tmdbFr_explode.merge(movieFranceV2Explode, on="tconst")

In [31]:
# Filtrer le full_data sur les films lancés
full_data.query('status == "Released"', inplace=True)

In [32]:
# Afficher full_data
full_data

,tconst,category,primaryName,averageRating_x,numVotes_x,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,startYear,runtimeMinutes,genres,averageRating_y,numVotes_y,ordering,title
0,tt0308078,director,Jean Epstein,6.1,16,0,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,Ciné-Liberté,2016,48,Documentary,6.1,16,1,Les bâtisseurs
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Comedy,7.0,33935,27,8 femmes
2,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Crime,7.0,33935,27,8 femmes
3,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Musical,7.0,33935,27,8 femmes
4,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Fidélité Productions,2002,111,Comedy,7.0,33935,27,8 femmes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823944,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Dryades Films,2018,64,Documentary,7.2,113,4,Coming Out
823945,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Upside Films,2018,64,Documentary,7.2,113,4,Coming Out
823946,tt9767986,director,Francis Amat,6.2,58,0,en,2.378,/yAuOJlMK2DvHEbvpsf2H71Ew8MD.jpg,2020-07-13,0,['en'],Released,Zatopeka Films,2019,103,Biography,6.2,58,2,Unraveling Athena
823947,tt9767986,director,Francis Amat,6.2,58,0,en,2.378,/yAuOJlMK2DvHEbvpsf2H71Ew8MD.jpg,2020-07-13,0,['en'],Released,Zatopeka Films,2019,103,Documentary,6.2,58,2,Unraveling Athena


In [33]:
# Factorisation des colonnes utiles et stocker dans de nouvelles colonnes
full_data[["category_fact", "primaryName_fact", "production_companies_name_fact", "genres_fact"]] = full_data[["category", "primaryName", "production_companies_name", "genres"]].apply(lambda column: pd.factorize(column)[0])

In [34]:
# Afficher le dataframe factorisé
full_data

,tconst,category,primaryName,averageRating_x,numVotes_x,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,startYear,runtimeMinutes,genres,averageRating_y,numVotes_y,ordering,title,category_fact,primaryName_fact,production_companies_name_fact,genres_fact
0,tt0308078,director,Jean Epstein,6.1,16,0,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,Ciné-Liberté,2016,48,Documentary,6.1,16,1,Les bâtisseurs,0,0,0,0
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Comedy,7.0,33935,27,8 femmes,1,1,1,1
2,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Crime,7.0,33935,27,8 femmes,1,1,1,2
3,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Musical,7.0,33935,27,8 femmes,1,1,1,3
4,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Fidélité Productions,2002,111,Comedy,7.0,33935,27,8 femmes,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823944,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Dryades Films,2018,64,Documentary,7.2,113,4,Coming Out,0,62646,22288,0
823945,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Upside Films,2018,64,Documentary,7.2,113,4,Coming Out,0,62646,13859,0
823946,tt9767986,director,Francis Amat,6.2,58,0,en,2.378,/yAuOJlMK2DvHEbvpsf2H71Ew8MD.jpg,2020-07-13,0,['en'],Released,Zatopeka Films,2019,103,Biography,6.2,58,2,Unraveling Athena,0,62647,22289,5
823947,tt9767986,director,Francis Amat,6.2,58,0,en,2.378,/yAuOJlMK2DvHEbvpsf2H71Ew8MD.jpg,2020-07-13,0,['en'],Released,Zatopeka Films,2019,103,Documentary,6.2,58,2,Unraveling Athena,0,62647,22289,0


In [35]:
# Drop les colonnes averageRating_x et numVotes_x , car en double et ordering car identifiant que l'on a autrement
full_data.drop(columns=["averageRating_x", "numVotes_x", "ordering"], inplace=True)

In [36]:
# Standardisation pour optimiser le futur modèle

## Step 3 : création du modèle

### Test 1

In [37]:
# Création des variables explicatives et expliquée
X = movieFrance_full_drop.select_dtypes("number")
y = movieFrance_full_drop["title"]

# Split pour avoir un set d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size = 0.75)

# Fit du modèle
modelNN = KNeighborsClassifier(n_neighbors=5).fit(X_train, y_train)

# Scores
#print("Score du modèle sur le set d'entraînement : ", modelNN.score(X_train, y_train))
#print("Score du modèle sur le set de test : ", modelNN.score(X_test, y_test))

In [38]:
# Test de prédicition avec des valeurs fixes
film_pref = "Avengers"
# actor_pref = "Mark Ruffalo"
# compagnie_pref = "Marvel Studios"
# real_pref = "Zack Snyder"
genre_pref = "Action"

# filmsDf = full_data.loc[(full_data["genres"] == genre_pref) | (full_data["primaryName"] == real_pref) | (full_data["primaryName"] == actor_pref) | (full_data["production_companies_name"] == compagnie_pref) | (full_data["title"] == film_pref), X.columns]
movieDf = movieFrance_full_drop.loc[(movieFrance_full_drop["genres"].str.contains(genre_pref)) | (movieFrance_full_drop["title"] == film_pref)]

In [39]:
movieDf

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
7,tt0113092,2000,100,"Action,Adventure,Drama",Action,Adventure,Drama,3.4,840,For the Cause,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,tt0119273,2000,88,"Action,Adventure,Animation",Action,Adventure,Animation,5.4,9170,Heavy Metal 2000,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,tt0120667,2005,106,"Action,Adventure,Fantasy",Action,Adventure,Fantasy,5.7,340682,Les 4 Fantastiques,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
23,tt0120737,2001,178,"Action,Adventure,Drama",Action,Adventure,Drama,8.8,1965944,Le Seigneur des anneaux : La Communauté de l'a...,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25,tt0120755,2000,123,"Action,Adventure,Thriller",Action,Adventure,Thriller,6.1,371556,Mission Impossible II,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29931,tt9893160,2022,89,"Action,Crime,Thriller",Action,Crime,Thriller,3.7,450,No Way Out,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
29935,tt9894470,2019,92,"Action,Crime,Horror",Action,Crime,Horror,6.1,9413,VFW,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
29938,tt9898858,2020,88,"Action,Comedy,Crime",Action,Comedy,Crime,5.2,14402,Coffee & Kareem,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29939,tt9900782,2019,145,"Action,Crime,Thriller",Action,Crime,Thriller,8.4,40629,Kaithi,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [40]:
distances, indices = modelNN.kneighbors(movieDf.select_dtypes("number"), n_neighbors=5)

In [41]:
sorted_indices = indices[np.argsort(-distances)]
sorted_distances = np.sort(-distances)

In [42]:
for i in range(0, len(sorted_indices[0])):
    print(full_data["title"].iloc[sorted_indices[0][i]])

13547           Désir
9416     80 for Brady
14143     La revanche
8025        Max Payne
22293           Chico
Name: title, dtype: object
17560    Gosford Park
2385      Van Helsing
14457     Le pianiste
15222      Sweet Land
10388       Braquages
Name: title, dtype: object
22350                   Détention
6726         Une si belle famille
9825               Lulu femme nue
10776    Bienvenue dans la jungle
12088         La famille suricate
Name: title, dtype: object
3115          Eddie the Eagle
7460             La locataire
667     Je rentre à la maison
7043        Super Héros Movie
4477    The Last Days on Mars
Name: title, dtype: object
1955                    In the Land of Women
4624                               Spotlight
17120            Je préfère qu'on reste amis
14103    Le chien, le général et les oiseaux
8162                               Elsewhere
Name: title, dtype: object


In [43]:
voisins = modelNN.kneighbors(movieDf.select_dtypes("number"), n_neighbors=5, return_distance=False)

In [44]:
inde_voisins = voisins[0]

In [45]:
voisins_proches = movieFrance_full_drop.iloc[inde_voisins]

In [46]:
print("Films les plus proches selon vos critères : ", list(voisins_proches["title"]))

Films les plus proches selon vos critères :  ['Mission Alcatraz', 'Les métamorphoses du choeur', 'The Ambassador', 'GasLand', 'Des vacances de Princesse: Bienvenue à Monte-Carlo']


In [47]:
full_data

,tconst,category,primaryName,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,startYear,runtimeMinutes,genres,averageRating_y,numVotes_y,title,category_fact,primaryName_fact,production_companies_name_fact,genres_fact
0,tt0308078,director,Jean Epstein,0,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,Ciné-Liberté,2016,48,Documentary,6.1,16,Les bâtisseurs,0,0,0,0
1,tt0283832,actress,Danielle Darrieux,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Comedy,7.0,33935,8 femmes,1,1,1,1
2,tt0283832,actress,Danielle Darrieux,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Crime,7.0,33935,8 femmes,1,1,1,2
3,tt0283832,actress,Danielle Darrieux,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,Musical,7.0,33935,8 femmes,1,1,1,3
4,tt0283832,actress,Danielle Darrieux,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Fidélité Productions,2002,111,Comedy,7.0,33935,8 femmes,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823944,tt9691896,director,Denis Parrot,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Dryades Films,2018,64,Documentary,7.2,113,Coming Out,0,62646,22288,0
823945,tt9691896,director,Denis Parrot,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Upside Films,2018,64,Documentary,7.2,113,Coming Out,0,62646,13859,0
823946,tt9767986,director,Francis Amat,0,en,2.378,/yAuOJlMK2DvHEbvpsf2H71Ew8MD.jpg,2020-07-13,0,['en'],Released,Zatopeka Films,2019,103,Biography,6.2,58,Unraveling Athena,0,62647,22289,5
823947,tt9767986,director,Francis Amat,0,en,2.378,/yAuOJlMK2DvHEbvpsf2H71Ew8MD.jpg,2020-07-13,0,['en'],Released,Zatopeka Films,2019,103,Documentary,6.2,58,Unraveling Athena,0,62647,22289,0


In [48]:
# Test avec NearestNeighbors
Xn = movieDf.select_dtypes("number")

modelNN2 = NearestNeighbors(n_neighbors=5)
modelNN2.fit(Xn)

NearestNeighbors()

In [49]:
voisins2 = modelNN2.kneighbors(movieDf.select_dtypes("number"), n_neighbors=5, return_distance=False)
ind_voisin = voisins2[0]
closest_film2 = movieFrance_full_drop.iloc[ind_voisin]
print(list(closest_film2["title"]))

['Histoire de la guerre civile', 'Ordinary Decent Criminal', 'Love Liza', "L'ascenseur (niveau 2)", 'Impostor']


Ici, le modèle ne renvoie pas des films qui ont réellement une similitude

### Test 2

In [50]:
# Table à utiliser
display(movieFrance_full_drop)

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
# Critères de sélection
pref_movie = "Spider-Man"
pref_genre = "Action"

# filmsDf = full_data.loc[(full_data["genres"] == genre_pref) | (full_data["primaryName"] == real_pref) | (full_data["primaryName"] == actor_pref) | (full_data["production_companies_name"] == compagnie_pref) | (full_data["title"] == film_pref), X.columns]
movieCriteria = movieFrance_full_drop.loc[((movieFrance_full_drop["genres"].str.contains(pref_genre)) | (movieFrance_full_drop["title"].str.contains(pref_movie))) & (movieFrance_full_drop["title"] != pref_movie)]

In [52]:
X_test2 = movieCriteria.select_dtypes("number")

In [53]:
modelNN = NearestNeighbors(n_neighbors=5, radius=0.4, algorithm="ball_tree", metric="minkowski")
modelNN.fit(X_test2)

NearestNeighbors(algorithm='ball_tree', radius=0.4)

In [54]:
movie_to_recommand = movieFrance_full_drop[movieFrance_full_drop["title"]  == pref_movie]

In [55]:
movie_to_recommand

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
66,tt0145487,2002,121,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,7.4,862824,Spider-Man,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [56]:
movieFromCriteria = movie_to_recommand.select_dtypes("number")
k_neighbors = modelNN.kneighbors(movieFromCriteria.select_dtypes("number"), n_neighbors=5, return_distance=False)
print(k_neighbors)

k_closest_movie_ind = k_neighbors[0]
print(k_closest_movie_ind)
k_closest_movies = movieCriteria["title"].iloc[k_closest_movie_ind]
print(k_closest_movies)

print("Voici la liste des films que nous vous recommandons selon vos critères : " ,list(k_closest_movies))

[[1573 2297  662 1285  775]]
[1573 2297  662 1285  775]
10308                        Iron Man 2
14869                      The Revenant
4153                                300
8390            Spider-Man: No Way Home
5019     Captain America: First Avenger
Name: title, dtype: object
Voici la liste des films que nous vous recommandons selon vos critères :  ['Iron Man 2', 'The Revenant', '300', 'Spider-Man: No Way Home', 'Captain America: First Avenger']


Ici le modèle propose des films qui sont plus cohérents. Essayons d'améliorer le modèle avec plusieurs entrées

In [57]:
# Test avec des changements de paramètres
modelNN3 = NearestNeighbors(n_neighbors=5, radius=0.4, algorithm="brute", metric="cosine")
modelNN3.fit(X_test2)

NearestNeighbors(algorithm='brute', metric='cosine', radius=0.4)

In [58]:
movieFromCriteria = movie_to_recommand.select_dtypes("number")
k_neighbors = modelNN3.kneighbors(movieFromCriteria.select_dtypes("number"), n_neighbors=5, return_distance=False)
print(k_neighbors)

k_closest_movie_ind = k_neighbors[0]
print(k_closest_movie_ind)
k_closest_movies = movieCriteria["title"].iloc[k_closest_movie_ind]
print(k_closest_movies)

print("Voici la liste des films que nous vous recommandons selon vos critères : " ,list(k_closest_movies))

[[1573  662  775 2461 1704]]
[1573  662  775 2461 1704]
10308                               Iron Man 2
4153                                       300
5019            Captain America: First Avenger
16051    Captain America: Le soldat de l'hiver
11129                               Iron Man 3
Name: title, dtype: object
Voici la liste des films que nous vous recommandons selon vos critères :  ['Iron Man 2', '300', 'Captain America: First Avenger', "Captain America: Le soldat de l'hiver", 'Iron Man 3']


In [59]:
index_pref_movie = movie_to_recommand.index[0]

In [60]:
index_pref_movie

66

In [61]:
distances, indices = modelNN3.kneighbors(movie_to_recommand.select_dtypes("number"), n_neighbors=5)

In [62]:
indices

array([[1573,  662,  775, 2461, 1704]])

In [63]:
sorted_distances = np.sort(distances)
sorted_distances

array([[4.23232671e-10, 5.39709610e-10, 9.87546045e-10, 1.27946398e-09,
        1.36950229e-09]])

Test avec KNeighborsClassifier

In [64]:
X = movieFrance_full_drop.drop("title", axis = 1).select_dtypes("number")
y = movieFrance_full_drop["title"]

# Split pour avoir un set d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size = 0.75)

modelKNN = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)

# Scores
# print("Score du modèle sur le set d'entraînement : ", modelKNN.score(X_train, y_train))
# print("Score du modèle sur le set de test : ", modelKNN.score(X_test, y_test))

In [65]:
new_movieFromCriteria = movie_to_recommand.select_dtypes("number")

new_neighbors = modelKNN.kneighbors(moviesFromCriteria.select_dtypes("number"), n_neighbors=5, return_distance=False)
new_neighbors_ind = new_neighbors[0]
print(new_neighbors_ind)
closest_movies_test2 = movieCriteria["title"].iloc[new_neighbors_ind]

print(list(closest_movies_test2))

NameError: name 'moviesFromCriteria' is not defined

Pas concluant

Après les tests, faisons un code clean

In [66]:
movieFrance_final = movieFrance.drop(columns=["Unnamed: 0", "titleType", "primaryTitle", "originalTitle", "isAdult", "region", "language", "isOriginalTitle", "ordering"])

In [67]:
movieFrance_full_drop

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
# movieFrance_final["genres"] = pd.factorize(movieFrance_final["genres"])[0]

In [69]:
filmPrefere = "Shrek"

In [70]:
genreFilmPrefere = movieFrance_full_drop[["Genre1", "Genre2", "Genre3"]].loc[movieFrance_final["title"]==filmPrefere].values[0]

In [71]:
genreFilmPrefere

array(['Adventure', 'Animation', 'Comedy'], dtype=object)

In [72]:
# DF avec les films qui ont le même genre que le film préféré, sans ce film ((movieFrance_full_drop["Genre1"]==genreFilmPrefere))
dfSansFilmPrefere = movieFrance_full_drop.loc[(movieFrance_full_drop["title"] != filmPrefere)]

In [73]:
dfSansFilmPrefere

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dfSansFilmPrefereStandardize = scaler.fit_transform(dfSansFilmPrefere.select_dtypes("number"))
dfSansFilmPrefereStandardize = pd.DataFrame(dfSansFilmPrefereStandardize, columns=movieFrance_full_drop.select_dtypes("number").columns)

In [75]:
dfSansFilmPrefereStandardize

,startYear,runtimeMinutes,averageRating,numVotes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1.115707,-0.026657,0.594567,-0.260483,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,2.401376,-1.037325,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
1,-1.889934,0.069316,0.254863,0.708172,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,1.681472,-0.360587,-0.416428,-1.037325,-0.226950,4.551693,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,2.729146,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
2,0.965425,-0.122629,0.254863,-0.259153,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
3,0.664861,0.085311,0.509641,-0.174274,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
4,-0.837960,-0.026657,-0.169767,-0.241810,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,-1.037325,-0.226950,-0.219698,-0.20001,3.010566,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29944,0.965425,0.009333,1.189048,-0.225918,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,1.681472,-0.360587,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
29945,0.815143,-0.102635,0.509641,-0.260603,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,2.401376,-1.037325,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
29946,0.815143,0.137297,1.104122,-0.260483,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,2.773255,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
29947,0.815143,0.021330,0.934270,-0.260724,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,-1.037325,4.406261,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598


In [76]:
X = dfSansFilmPrefereStandardize

In [77]:
X

,startYear,runtimeMinutes,averageRating,numVotes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1.115707,-0.026657,0.594567,-0.260483,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,2.401376,-1.037325,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
1,-1.889934,0.069316,0.254863,0.708172,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,1.681472,-0.360587,-0.416428,-1.037325,-0.226950,4.551693,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,2.729146,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
2,0.965425,-0.122629,0.254863,-0.259153,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
3,0.664861,0.085311,0.509641,-0.174274,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
4,-0.837960,-0.026657,-0.169767,-0.241810,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,-1.037325,-0.226950,-0.219698,-0.20001,3.010566,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29944,0.965425,0.009333,1.189048,-0.225918,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,1.681472,-0.360587,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
29945,0.815143,-0.102635,0.509641,-0.260603,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,2.401376,-1.037325,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
29946,0.815143,0.137297,1.104122,-0.260483,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,2.773255,-0.416428,0.964018,-0.226950,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598
29947,0.815143,0.021330,0.934270,-0.260724,-0.414826,-0.005779,-0.327496,-0.217687,-0.24038,-0.594717,-0.360587,-0.416428,-1.037325,4.406261,-0.219698,-0.20001,-0.332163,-0.167571,-0.095558,-0.268149,-0.03657,-0.366415,-0.202525,-0.124205,-0.404655,-0.133187,-0.059598


In [78]:
modelNN = NearestNeighbors(n_neighbors=5, algorithm="auto", metric="euclidean").fit(X)

In [79]:
filmPrefereDf = movieFrance_full_drop.loc[movieFrance_full_drop["title"] == filmPrefere, X.columns]
voisins = modelNN.kneighbors(filmPrefereDf, return_distance=False)

plusProcheVoisinsInd = voisins[0]
plusProchesFilms = dfSansFilmPrefere.iloc[plusProcheVoisinsInd]

print(list(plusProchesFilms["title"]))

['The Dark Knight : Le Chevalier noir', 'Inception', 'Interstellar', "Le Seigneur des anneaux : La Communauté de l'anneau", 'Le Seigneur des anneaux : Le Retour du roi']


In [80]:
filmPrefereDf

,startYear,runtimeMinutes,averageRating,numVotes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
39,2001,90,7.9,719071,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Test 3

In [81]:
dfSansFilmPrefere

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [82]:
filmPrefere = "Shrek"
dfSansFilmPrefere = movieFrance_full_drop.loc[(movieFrance_full_drop["title"] != filmPrefere)]
# scaler = StandardScaler()
# dfSansFilmPrefereStandardize = scaler.fit_transform(dfSansFilmPrefere.select_dtypes("number"))
# dfSansFilmPrefereStandardize = pd.DataFrame(dfSansFilmPrefereStandardize, columns=movieFrance_full_drop.select_dtypes("number").columns)

X = dfSansFilmPrefere.select_dtypes("number")
y = dfSansFilmPrefere["title"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = KNeighborsClassifier()
model.fit(X, y)

# Get the index of the preferred movie
index = dfSansFilmPrefere[dfSansFilmPrefere["title"] == filmPrefere].index[0]

# Get the 5 most similar movies
distances, indices = model.kneighbors([X_train.loc[index]], n_neighbors=6)

# Print the titles of the recommended movies
for i in indices[0][1:]:
    print(dfSansFilmPrefere.loc[i, 'title'])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [83]:
query_index = np.random.choice(movieFrance_full_drop.shape[0])
print("Choosen Movie is: ",movieFrance_full_drop.index[query_index])

Choosen Movie is:  26845


In [84]:
movieFrance_full_drop.loc[movieFrance_full_drop["title"] == filmPrefere].index[0]

39

Test sans les données standardisées

In [85]:
from scipy.sparse import csr_matrix

filmPrefere = "Avengers"

index = movieFrance_full_drop.loc[movieFrance_full_drop["title"] == filmPrefere].index[0]

matrix = csr_matrix(movieFrance_full_drop.select_dtypes("number"))

model = NearestNeighbors(metric="cosine", algorithm="brute")

model.fit(matrix)

distances, indices = model.kneighbors(movieFrance_full_drop.select_dtypes("number").iloc[index, :].values.reshape(1,-1), n_neighbors=6)

In [86]:
movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(movieFrance_full_drop.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='movie')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(movieFrance_full_drop[movieFrance_full_drop.index == index]["title"].values[0]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, movieFrance_full_drop[movieFrance_full_drop.index == recommend["movie"].iloc[i]]["title"].values[0], recommend["distance"].iloc[i]))

Recommendations for Avengers:

0: Avatar, with distance of 2.9246666199966853e-09
1: Les infiltrés, with distance of 9.391037947281689e-10
2: Le Prestige, with distance of 3.4552760652672987e-10
3: Shutter Island, with distance of 2.833414614045182e-10
4: Joker, with distance of 2.2493995555095125e-10


Test avec les données standardisées

Test avec des filtres sur les acteurs, réalisateurs et maison de production

In [87]:
all_merge = d_a_tmdbFr_explode.merge(movieFrance_full_drop, how="inner", on="tconst")

In [88]:
all_merge["genres"] all_merge["genres"].str.split(",").explode()

SyntaxError: invalid syntax (1089138260.py, line 1)

In [89]:
# Filtres sur les maisons de production/acteurs/actrice

maison_prod = "Marvel Studios"
acteur = "Chris Evans"
producteur = "Zack Snyder"

mask_maison_prod = all_merge["production_companies_name"] == maison_prod
mask_acteur_producteur = ((all_merge["primaryName"] == acteur) | (all_merge["primaryName"] == producteur))
all_merge_filter = all_merge.loc[(mask_acteur_producteur) | (mask_maison_prod)]

In [90]:
# Dataframe des films en tenant compte des critères de sélection
movie_filter = movieFrance_full_drop.loc[movieFrance_full_drop["tconst"].isin(all_merge_filter["tconst"])]

In [91]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dfStandard = scaler.fit_transform(movieFrance_full_drop.select_dtypes("number"))
dfStandard = pd.DataFrame(dfStandard, columns=movieFrance_full_drop.select_dtypes("number").columns)

In [92]:
from scipy.sparse import csr_matrix

# Demande le film préféré
filmPrefere = "Avengers"

# Garde en mémoire l'index du film préféré
index = movieFrance_full_drop.loc[movieFrance_full_drop["title"] == filmPrefere].index[0]

# Création d'une matrice sur le dataframe normalisé
matrix = csr_matrix(dfStandard)

# Création du modéle avec les paramètres souhaité
model = NearestNeighbors(metric="cosine", algorithm="brute")

# Entraînement du modèle
model.fit(matrix)

# Garde en mémoire les distances et indexes des 20 films les plus proches
distances, indices = model.kneighbors(dfStandard.iloc[index, :].values.reshape(1,-1), n_neighbors=21)

In [93]:
# Création de listes pour stocker les films et les distances
movie = []
distance = []

# Boucle et aplatissement de la liste distances (évite d'avoir des sous_listes)
for i in range(0, len(distances.flatten())):

    # Si la distance est différente de 0
    if i != 0:

        # J'ajoute l'index du film dans la liste movieet la distance dans la liste distance
        movie.append(movieFrance_full_drop.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

# Transformation en Séries pandas
m=pd.Series(movie,name='movie')
d=pd.Series(distance,name='distance')

# Concaténation des séries pour faire un dataframe
recommend = pd.concat([m,d], axis=1)

# Trie des valeurs sur les distances par ordre croissant 
recommend = recommend.sort_values('distance',ascending=True)

# Filtrer les films recommandés avec les critères de préférences
recommend_filter = recommend.loc[recommend["movie"].isin(movie_filter.index)]

# Affichage du film à recommander
print('Recommendations pour le film {0}:\n'.format(movieFrance_full_drop[movieFrance_full_drop.index == index]["title"].values[0]))

# Boucle pour afficher chaque film à recommander
for i in range(0,recommend_filter.shape[0]):
    print('{0}: {1}, avec une distance de {2}'.format(i, movieFrance_full_drop[movieFrance_full_drop.index == recommend_filter["movie"].iloc[i]]["title"].values[0], recommend["distance"].iloc[i]))

Recommendations pour le film Avengers:

0: Iron Man 2, avec une distance de 0.015011331797507466
1: Captain America: Civil War, avec une distance de 0.01864214771121675
2: Avengers: Infinity War, avec une distance de 0.022561636464501933
3: Iron Man, avec une distance de 0.02551639394914118
4: Avengers : L'Ère d'Ultron, avec une distance de 0.03271800032786054
5: Iron Man 3, avec une distance de 0.034254671792881
6: Captain America: Le soldat de l'hiver, avec une distance de 0.03565382848453913
7: Captain America: First Avenger, avec une distance de 0.03891314349412445


### Modèle fonctionnel

In [94]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dfStandard = scaler.fit_transform(movieFrance_full_drop.select_dtypes("number"))
dfStandard = pd.DataFrame(dfStandard, columns=movieFrance_full_drop.select_dtypes("number").columns)

In [95]:
from scipy.sparse import csr_matrix

# Demande le film préféré
filmPrefere = "Avengers"

# Garde en mémoire l'index du film préféré
index = movieFrance_full_drop.loc[movieFrance_full_drop["title"] == filmPrefere].index[0]

# Création d'une matrice sur le dataframe normalisé
matrix = csr_matrix(dfStandard)

# Création du modéle avec les paramètres souhaité
model = NearestNeighbors(metric="cosine", algorithm="brute")

# Entraînement du modèle
model.fit(matrix)

# Garde en mémoire les distances et indexes des 20 films les plus proches
distances, indices = model.kneighbors(dfStandard.iloc[index, :].values.reshape(1,-1), n_neighbors=21)

In [96]:
# Création de listes pour stocker les films et les distances
movie = []
distance = []

# Boucle et aplatissement de la liste distances (évite d'avoir des sous_listes)
for i in range(0, len(distances.flatten())):

    # Si la distance est différente de 0
    if i != 0:

        # J'ajoute l'index du film dans la liste movieet la distance dans la liste distance
        movie.append(movieFrance_full_drop.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

# Transformation en Séries pandas
m=pd.Series(movie,name='movie')
d=pd.Series(distance,name='distance')

# Concaténation des séries pour faire un dataframe
recommend = pd.concat([m,d], axis=1)

# Trie des valeurs sur les distances par ordre croissant 
recommend = recommend.sort_values('distance',ascending=True)

# Affichage du film à recommander
print('Recommendations pour le film {0}:\n'.format(movieFrance_full_drop[movieFrance_full_drop.index == index]["title"].values[0]))

# Boucle pour afficher chaque film à recommander
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, avec une distance de {2}'.format(i, movieFrance_full_drop[movieFrance_full_drop.index == recommend["movie"].iloc[i]]["title"].values[0], recommend["distance"].iloc[i]))

Recommendations pour le film Avengers:

0: V pour vendetta, avec une distance de 0.015011331797507466
1: Inception, avec une distance de 0.01864214771121675
2: Iron Man 2, avec une distance de 0.022561636464501933
3: Captain America: Civil War, avec une distance de 0.02551639394914118
4: Avengers: Infinity War, avec une distance de 0.03271800032786054
5: Interstellar, avec une distance de 0.034254671792881
6: Iron Man, avec une distance de 0.03565382848453913
7: Mad Max: Fury Road, avec une distance de 0.03891314349412445
8: Logan, avec une distance de 0.04147157056884809
9: X-Men : Le Commencement, avec une distance de 0.04354899334051576
10: Hunger Games, avec une distance de 0.04626765194573079
11: Star Wars : Épisode VII - Le Réveil de la Force, avec une distance de 0.04876342622187291
12: Le Prestige, avec une distance de 0.05216357835398
13: Avengers : L'Ère d'Ultron, avec une distance de 0.05596386777114104
14: Iron Man 3, avec une distance de 0.058751101697973596
15: Captain Am

### Combinaison des deux modèles dans le cas où les filtres appliqués ne permettent pas d'avoir au moins 5 recommandations 

In [97]:
# Filtres sur les maisons de production/acteurs/actrice

maison_prod = "Marvel Studios"
acteur = "Chris Evans"
producteur = "Zack Snyder"
mask_maison_prod = all_merge["production_companies_name"] == maison_prod
mask_acteur_producteur = ((all_merge["primaryName"] == acteur) | (all_merge["primaryName"] == producteur))
all_merge_filter = all_merge.loc[(mask_acteur_producteur) | (mask_maison_prod)]

# Demande le film préféré
filmPrefere = "Shrek"

# Dataframe des films en tenant compte des critères de sélection
movie_filter = movieFrance_full_drop.loc[movieFrance_full_drop["tconst"].isin(all_merge_filter["tconst"])]

# Standardiser les valeurs
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dfStandard = scaler.fit_transform(movieFrance_full_drop.select_dtypes("number"))
dfStandard = pd.DataFrame(dfStandard, columns=movieFrance_full_drop.select_dtypes("number").columns)

In [98]:
from scipy.sparse import csr_matrix

# Garde en mémoire l'index du film préféré
index = movieFrance_full_drop.loc[movieFrance_full_drop["title"] == filmPrefere].index[0]

# Création d'une matrice sur le dataframe normalisé
matrix = csr_matrix(dfStandard)

# Création du modéle avec les paramètres souhaité
model = NearestNeighbors(metric="cosine", algorithm="brute")

# Entraînement du modèle
model.fit(matrix)

# Garde en mémoire les distances et indexes des 20 films les plus proches
distances, indices = model.kneighbors(dfStandard.iloc[index, :].values.reshape(1,-1), n_neighbors=21)

In [99]:
# Création de listes pour stocker les films et les distances
movie = []
distance = []

# Boucle et aplatissement de la liste distances (évite d'avoir des sous_listes)
for i in range(0, len(distances.flatten())):

    # Si la distance est différente de 0
    if i != 0:

        # J'ajoute l'index du film dans la liste movieet la distance dans la liste distance
        movie.append(movieFrance_full_drop.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

# Transformation en Séries pandas
m=pd.Series(movie,name='movie')
d=pd.Series(distance,name='distance')

# Concaténation des séries pour faire un dataframe
recommend = pd.concat([m,d], axis=1)

# Trie des valeurs sur les distances par ordre croissant 
recommend = recommend.sort_values('distance',ascending=True)

# Filtrer les films recommandés avec les critères de préférences
recommend_filter = recommend.loc[recommend["movie"].isin(movie_filter.index)]

# Affichage du film à recommander
print('Recommendations pour le film {0}:\n'.format(movieFrance_full_drop[movieFrance_full_drop.index == index]["title"].values[0]))

# Conditions pour éviter de n'avoir aucun affichage de films avec mes filtres ou moins de 5
if recommend_filter.shape[0] > 5:
    # Boucle pour afficher chaque film à recommander filtrés
    for i in range(0,recommend_filter.shape[0]):
        print('{0}: {1}, avec une distance de {2}'.format(i, movieFrance_full_drop[movieFrance_full_drop.index == recommend_filter["movie"].iloc[i]]["title"].values[0], recommend_filter["distance"].iloc[i]))
else:
    # Boucle pour afficher chaque film à recommander non filtré
    for i in range(0,recommend.shape[0]):
        print('{0}: {1}, avec une distance de {2}'.format(i, movieFrance_full_drop[movieFrance_full_drop.index == recommend["movie"].iloc[i]]["title"].values[0], recommend["distance"].iloc[i]))

Recommendations pour le film Shrek:

0: Ratatouille, avec une distance de 0.006099677394929226
1: Monstres & Cie, avec une distance de 0.009654807053853576
2: Moi, moche et méchant, avec une distance de 0.014632409044498473
3: Toy Story 3, avec une distance de 0.014711874273168335
4: L'Âge de glace, avec une distance de 0.01525611281286654
5: Shrek 2, avec une distance de 0.018063094180352568
6: La reine des neiges, avec une distance de 0.01811226511635733
7: Le Monde de Nemo, avec une distance de 0.020625313485089003
8: Vice-versa, avec une distance de 0.022756014155182358
9: Raiponce, avec une distance de 0.027561936645021268
10: Cars - Quatre roues, avec une distance de 0.027716810784962354
11: Là-haut, avec une distance de 0.028327038019562822
12: Madagascar, avec une distance de 0.035969431369674565
13: Zootopie, avec une distance de 0.03919373272663007
14: Les mondes de Ralph, avec une distance de 0.04082963198057199
15: Moi, moche et méchant 2, avec une distance de 0.05311288891

In [100]:
movieFrance_full_drop

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Poster path concatenate

In [101]:
recommend

,movie,distance
0,3534,0.006100
1,315,0.009655
2,11419,0.014632
3,4581,0.014712
4,1322,0.015256
5,1976,0.018063
6,18380,0.018112
7,1290,0.020625
8,17297,0.022756
9,3813,0.027562


In [102]:
df_recommendation = movieFrance_full_drop.loc[movieFrance_full_drop.index.isin(recommend["movie"])]

In [103]:
mapping_series = all_merge.set_index('tconst')['poster_path']

In [104]:
df_poster_path = mapping_series.to_frame().reset_index()
df_poster_path = df_poster_path.drop_duplicates(subset=["tconst"])

In [105]:
# Merge de df_recommandation et df_poster_path
df_poster_path_merge = df_poster_path.merge(df_recommendation, how="inner", on="tconst")

In [106]:
df_final = pd.concat([df_poster_path_merge, recommend], axis=1)

In [122]:
movieFrance_full_drop

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [121]:
all_merge[["tconst", "poster_path"]].drop_duplicates("tconst").merge(movieFrance_full_drop, how="inner", on="tconst").to_csv("movieFrance_full_dropV2.csv")

,tconst,poster_path,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0308078,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,2016,48,Documentary,Documentary,NaN,NaN,6.1,16,Les bâtisseurs,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0283832,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002,111,"Comedy,Crime,Musical",Comedy,Crime,Musical,7.0,33935,8 femmes,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,tt0487469,/xgNkIVuTXZke77VGklducGMPDL6.jpg,2006,87,Comedy,Comedy,NaN,NaN,5.6,164,Nouvelle chance,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0808417,/aU8i2QAdTyRR1nYb36Gq51xXP8p.jpg,2007,96,"Animation,Biography,Drama",Animation,Biography,Drama,8.0,98867,Persepolis,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0889606,/qr5XXlIZ1egIBqmESkCmNCsrJmk.jpg,2007,104,"Comedy,Mystery",Comedy,Mystery,NaN,5.8,675,L'heure zéro,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26698,tt9617464,/5L576j6jxvWmdgA94xfuM7gCoGz.jpg,2019,76,Documentary,Documentary,NaN,NaN,7.5,31,Museum Town,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26699,tt9654050,/dqXbYgaXOtZxIRNE6dS9Nynvw2r.jpg,2019,96,"Biography,Documentary",Biography,Documentary,NaN,6.6,332,Lemebel,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26700,tt9658178,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019,93,Documentary,Documentary,NaN,NaN,7.4,845,Talking About Trees,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26701,tt9691896,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2018,64,Documentary,Documentary,NaN,NaN,7.2,113,Coming Out,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
all_merge

,tconst,category,primaryName,averageRating_x,numVotes_x,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating_y,numVotes_y,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0308078,director,Jean Epstein,6.1,16,0,fr,0.600,/3P7pZsvfaEoj6LScAZ2CgX6MBRr.jpg,1938-01-01,0,['fr'],Released,Ciné-Liberté,2016,48,Documentary,Documentary,NaN,NaN,6.1,16,Les bâtisseurs,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,France 2 Cinéma,2002,111,"Comedy,Crime,Musical",Comedy,Crime,Musical,7.0,33935,8 femmes,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Fidélité Productions,2002,111,"Comedy,Crime,Musical",Comedy,Crime,Musical,7.0,33935,8 femmes,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,Le Studio Canal+,2002,111,"Comedy,Crime,Musical",Comedy,Crime,Musical,7.0,33935,8 femmes,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,tt0283832,actress,Danielle Darrieux,7.0,33935,8000000,fr,10.001,/bJu4Gm7NQb2woPiIovrPD5jNWzf.jpg,2002-02-06,0,"['en', 'fr']",Released,CNC,2002,111,"Comedy,Crime,Musical",Comedy,Crime,Musical,7.0,33935,8 femmes,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361668,tt9658178,director,Suhaib Gasmelbari,7.4,845,0,en,3.063,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019-12-18,0,"['ar', 'en']",Released,SFG,2019,93,Documentary,Documentary,NaN,NaN,7.4,845,Talking About Trees,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361669,tt9658178,director,Suhaib Gasmelbari,7.4,845,0,en,3.063,/yekoa8YOzD1biPudP4SxHiXyp7Y.jpg,2019-12-18,0,"['ar', 'en']",Released,Vidéo de Poche,2019,93,Documentary,Documentary,NaN,NaN,7.4,845,Talking About Trees,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361670,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Dryades Films,2018,64,Documentary,Documentary,NaN,NaN,7.2,113,Coming Out,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361671,tt9691896,director,Denis Parrot,7.2,113,0,fr,1.336,/fX39zTZ4mpnfWywOrgJj3AFGtF.jpg,2019-05-01,0,"['en', 'fr', 'de', 'ja', 'ru']",Released,Upside Films,2018,64,Documentary,Documentary,NaN,NaN,7.2,113,Coming Out,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [117]:
movieFrance_full_drop.loc[movieFrance_full_drop["title"] == "Iron Man"]

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
3273,tt0371746,2008,126,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,7.9,1107275,Iron Man,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [120]:
all_merge.loc[all_merge["tconst"] == "tt0371746"].drop_duplicates("tconst")

,tconst,category,primaryName,averageRating_x,numVotes_x,budget,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating_y,numVotes_y,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
46547,tt0371746,actor,Jeff Bridges,7.9,1107275,140000000,en,78.14,/78lPtwv72eTNqFW9COBYI0dWDJa.jpg,2008-04-30,585174222,"['en', 'fa', 'ur', 'ar']",Released,Marvel Studios,2008,126,"Action,Adventure,Sci-Fi",Action,Adventure,Sci-Fi,7.9,1107275,Iron Man,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [116]:
tmdbFr

,Unnamed: 0,backdrop_path,budget,id,imdb_id,original_language,popularity,poster_path,release_date,revenue,spoken_languages,status,production_companies_name,production_companies_country
0,10,/yzqaKAhglTrkeOfuIXYYArf0WnA.jpg,94000000,12,tt0266543,en,68.898,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,2003-05-30,940335536,['en'],Released,['Pixar'],['US']
1,14,/ikR2qy9xJCHX7M8i5rcvuNfdYXs.jpg,12800000,16,tt0168629,en,13.781,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,2000-06-30,40031879,['en'],Released,"['Lantia Cinema & Audiovisivi', 'Fine Line Fea...","['', '', 'DK', 'DK', 'SE', 'FR', 'SE', 'FR', '..."
2,15,/dhr0q4eiRr8ltqPig32TwhPRdaD.jpg,0,17,tt0411267,en,7.178,/wZeBHVnCvaS2bwkb8jFQ0PwZwXq.jpg,2005-09-28,6593579,"['en', 'cy']",Released,"['Constantin Film', 'Impact Pictures', 'Isle o...","['DE', 'GB', 'GB', 'GB']"
3,18,/kZyurQjTMLHalUxs7sHgH5XeiwO.jpg,2500000,20,tt0314412,en,11.068,/sFSkn5rrQqXJkRNa2rMWqzmEuhR.jpg,2003-03-07,12300000,['en'],Released,"['El Deseo', 'Milestone Productions']","['ES', '']"
4,20,/uRNgkJSkNBFbbn9fPsEjDIy8Sh3.jpg,140000000,22,tt0325980,en,88.516,/z8onk7LV9Mmw6zKz4hT6pzzvmvl.jpg,2003-07-09,655011224,['en'],Released,"['Walt Disney Pictures', 'Jerry Bruckheimer Fi...","['US', 'US']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26893,308888,NaN,0,230680,tt0366808,de,0.840,NaN,2005-04-28,0,['de'],Released,"['Junifilm', 'Eikon Film']",NaN
26894,308919,NaN,0,230949,tt0855783,ku,0.600,/o1pTHdU3mCZ3yzEWYDvIDs7t0MD.jpg,2007-01-24,0,"['ku', 'tr']",Released,[],NaN
26895,309034,NaN,0,231386,tt0492996,pt,0.600,/goBEZTpXJJ3rshJcJJrx0x0qKAH.jpg,2003-01-01,0,['pt'],Released,[],NaN
26896,309276,NaN,0,232217,tt1596750,en,0.823,/8OUDkwXvwfs9C17MYoCdwmHE2h1.jpg,2008-11-20,0,[],Released,[],NaN


In [124]:
movieFrance_full_drop

,tconst,startYear,runtimeMinutes,genres,Genre1,Genre2,Genre3,averageRating,numVotes,title,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0013274,2021,94,Documentary,Documentary,NaN,NaN,6.8,62,Histoire de la guerre civile,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,tt0035423,2001,118,"Comedy,Fantasy,Romance",Comedy,Fantasy,Romance,6.4,88199,Kate et Léopold,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,tt0062336,2020,70,Drama,Drama,NaN,NaN,6.4,183,El Tango del Viudo y Su Espejo Deformante,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,tt0069049,2018,122,Drama,Drama,NaN,NaN,6.7,7906,De l'autre côté du vent,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,tt0082328,2008,94,Horror,Horror,NaN,NaN,5.9,1761,L'incarnation du démon,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29945,tt9911196,2020,103,"Comedy,Drama",Comedy,Drama,NaN,7.5,3207,De Beentjes van Sint-Hildegard,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29946,tt9913084,2019,75,Documentary,Documentary,NaN,NaN,6.7,51,Diabolik sono io,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29947,tt9913936,2019,135,"Crime,Drama",Crime,Drama,NaN,7.4,62,Le paradis de Diego,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29948,tt9915790,2019,106,Family,Family,NaN,NaN,7.2,40,Bobbyr Bondhura,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
